In [94]:
import pandas as pd
import json
import requests
url = 'https://raw.githubusercontent.com/brearenee/NLP-Project/main/dataset/StarTrekDialogue_v2.json'
response = requests.get(url)

##This CodeBlock is thanks to ChatGPT :-) 
if response.status_code == 200:
    json_data = json.loads(response.text)
    lines = []
    characters = []
    episodes = []
  
    # extract the information from the JSON file for the "TNG" series
    for series_name, series_data in json_data.items():
        if series_name == "TNG": 
            for episode_name, episode_data in series_data.items():
                for character_name, character_lines in episode_data.items():
                    for line_text in character_lines:
                        lines.append(line_text)
                        characters.append(character_name)
                        episodes.append(episode_name)
                     
    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'Line': lines,
        'Character': characters,
    })

    # Remove duplicate lines, keeping the first occurrence (preserving the original order)
    df = df.drop_duplicates(subset='Line', keep='first')

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    
##Remove Outliers (Characters with less than 1000 lines)
character_counts = df['Character'].value_counts()
characters_to_remove = character_counts[character_counts < 1000].index
df = df[~df['Character'].isin(characters_to_remove)]

##Print Value Count. 
print(df['Character'].value_counts())


Character
PICARD     10798
RIKER       6454
DATA        5699
LAFORGE     4111
WORF        3185
CRUSHER     2944
TROI        2856
WESLEY      1206
Name: count, dtype: int64


# BERT

In [95]:
#https://www.analyticsvidhya.com/blog/2021/12/multiclass-classification-using-transformers/


#Split the data 
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=20)

#Converting our Character column into Categorical data
encoded_dict = {'PICARD':0,'RIKER':1, 'DATA':2, 'LAFORGE':3, 
                'WORF':4, 'CRUSHER':5, 'TROI':6,'WESLEY':7}
train_df['Character'] = train_df.Character.map(encoded_dict)
val_df['Character'] = val_df.Character.map(encoded_dict)

print(train_df['Character'].value_counts())
print(val_df['Character'].value_counts())




Character
0    8711
1    5152
2    4580
3    3269
4    2535
5    2345
6    2267
7     943
Name: count, dtype: int64
Character
0    2087
1    1302
2    1119
3     842
4     650
5     599
6     589
7     263
Name: count, dtype: int64


In [96]:
val_df.head(20)

,Line,Character
20650,"Q, get to the controls or get the hell out of ...",3
20069,Why do you have all this anger toward me?,6
15012,"Interesting. Yes, that code hasn't been used i...",0
44403,No. Get ready to deploy the mines.,1
28557,Ben Maxwell? But he's one of Starfleet's fines...,0
30967,"Mister Barclay, I want you to stop this experi...",0
7991,Eighty five microvolts. Again.\r Ninety. Again...,5
39406,Forbid it?,0
39909,Picard to Bridge. Report.,0
45915,"What I mean is, it is not your fault.",4


In [97]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_df.Character)
y_test = to_categorical(val_df.Character)

#We have successfully processed our Character column( target); 
#now, it’s time to process our input text data using a tokenizer.

In [98]:
import transformers

#Loading Model and Tokenizer from the transformers package 

from transformers import AutoTokenizer,TFBertModel
#bert-base-uncased is another possible one
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
#TFBertModel = pretrained BERT model for Tensor Flow
bert = TFBertModel.from_pretrained('bert-base-uncased')

#Input Data Modeling

#Before training, we need to convert the input textual data into 
#BERT’s input data format using a tokenizer.
#Since we have loaded bert-base-cased, 
#so tokenizer will also be Bert-base-cased.
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=train_df.Line.tolist(),
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=val_df.Line.tolist(),
    add_special_tokens=True,
    max_length=40,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)


#Hereafter data modelling, the tokenizer will return a dictionary (x_train) containing ‘Input_ids’, ‘attention_mask’ as key for their respective
#data.

input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

# Model Building

In [99]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

max_len = 40
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(64,activation = 'relu')(out)
y = Dense(8,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

# Model Compilation

Defining learning parameters and compiling the model.

In [100]:

optimizer = tf.keras.optimizers.legacy.Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

# Model Training

In [101]:
#We have the model ready with x_train, y_train. You can now train the model.
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=2,
    batch_size=32
)


Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/backend.py:5562: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


932/932 [==============================] - 300s 297ms/step - loss: 1.5439 - balanced_accuracy: 0.4302 - val_loss: 1.4280 - val_balanced_accuracy: 0.4722
Epoch 2/2
932/932 [==============================] - 274s 294ms/step - loss: 1.3576 - balanced_accuracy: 0.5031 - val_loss: 1.3992 - val_balanced_accuracy: 0.4797


# Model Evaluation

In [102]:
import numpy as np
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = val_df.Character

from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))

233/233 [==============================] - 24s 88ms/step
              precision    recall  f1-score   support

           0       0.50      0.70      0.58      2087
           1       0.35      0.32      0.34      1302
           2       0.66      0.62      0.64      1119
           3       0.48      0.50      0.49       842
           4       0.46      0.42      0.44       650
           5       0.40      0.32      0.35       599
           6       0.38      0.22      0.28       589
           7       0.00      0.00      0.00       263

    accuracy                           0.48      7451
   macro avg       0.40      0.39      0.39      7451
weighted avg       0.46      0.48      0.46      7451



/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtyp

#1 1 epoch.  max_length = 70. second layer 34.  Accuracy = 42ish. 
#2 2 epoch. Max_length = 50? 40? second layer 64. Auucracy = .4797 